### Importando Librerías

In [79]:
import pandas as pd
import numpy as np

### Leo archivo p_data.csv y seteo el index por ID

In [80]:
dfp_Data = pd.read_csv('files/p_data.csv', sep=';')
dfp_Data = dfp_Data.set_index('ID')
# Limpio los valores nulos
dfp_Data = dfp_Data.dropna()
print(dfp_Data.shape)
dfp_Data.head()

(48841, 8)


,Age,Sex,Race,Country,Education,Education-Num,Marital Status,Relationship
ID,,,,,,,,
49084,39.0,Male,White,United-States,Bachelors,13.0,Never-married,Not-in-family
68553,50.0,Male,White,United-States,Bachelors,13.0,Married-civ-spouse,Husband
143654,38.0,Male,White,United-States,HS-grad,9.0,Divorced,Not-in-family
41510,53.0,Male,Black,United-States,11th,7.0,Married-civ-spouse,Husband
195025,28.0,Female,Black,Cuba,Bachelors,13.0,Married-civ-spouse,Wife


### Leo archivo w_data y seteo el index por ID

In [81]:
dfw_Data = pd.read_csv('files/w_data.csv', sep=';')
dfw_Data = dfw_Data.set_index('ID')
## Limpio los valores nulos
dfw_Data = dfw_Data.dropna()
print(dfw_Data.shape)
dfw_Data.head()

(48842, 6)


,Workclass,fnlwgt,Capital Gain,Capital Loss,Hours per week,Target
ID,,,,,,
49084,State-gov,77516.0,2174.0,0.0,40.0,<=50K
68553,Self-emp-not-inc,83311.0,0.0,0.0,13.0,<=50K
143654,Private,215646.0,0.0,0.0,40.0,<=50K
41510,Private,234721.0,0.0,0.0,40.0,<=50K
195025,Private,338409.0,0.0,0.0,40.0,<=50K


### Fusiono los dos data frames en función del ID

In [82]:
dfP_W = pd.merge(dfp_Data, dfw_Data, how='outer', on='ID')
print(dfP_W.shape)
dfP_W.head()

(48842, 14)


,Age,Sex,Race,Country,Education,Education-Num,Marital Status,Relationship,Workclass,fnlwgt,Capital Gain,Capital Loss,Hours per week,Target
ID,,,,,,,,,,,,,,
49084,39.0,Male,White,United-States,Bachelors,13.0,Never-married,Not-in-family,State-gov,77516.0,2174.0,0.0,40.0,<=50K
68553,50.0,Male,White,United-States,Bachelors,13.0,Married-civ-spouse,Husband,Self-emp-not-inc,83311.0,0.0,0.0,13.0,<=50K
143654,38.0,Male,White,United-States,HS-grad,9.0,Divorced,Not-in-family,Private,215646.0,0.0,0.0,40.0,<=50K
41510,53.0,Male,Black,United-States,11th,7.0,Married-civ-spouse,Husband,Private,234721.0,0.0,0.0,40.0,<=50K
195025,28.0,Female,Black,Cuba,Bachelors,13.0,Married-civ-spouse,Wife,Private,338409.0,0.0,0.0,40.0,<=50K


### 1.- Para saber cuantos ID tienen en comun y ver que registros son, verifico valores nulos dentro de los campos del primer dataset y luego del segundo

In [83]:
dfP_W[dfP_W['Age'].isnull()]

,Age,Sex,Race,Country,Education,Education-Num,Marital Status,Relationship,Workclass,fnlwgt,Capital Gain,Capital Loss,Hours per week,Target
ID,,,,,,,,,,,,,,
1344995623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Self-emp-inc,182148.0,0.0,0.0,60.0,>50K.


In [84]:
dfP_W[dfP_W['Workclass'].isnull()]

,Age,Sex,Race,Country,Education,Education-Num,Marital Status,Relationship,Workclass,fnlwgt,Capital Gain,Capital Loss,Hours per week,Target
ID,,,,,,,,,,,,,,


#### Se puede evidenciar que sólo hay un registro que no tienen en común, lo que indica que los que tienen en común son 48842 menos ese registro, 48841

### 2.- Para corregir los target, primero veo que valores hay para luego actualizar según cierta lógica

In [85]:
dfP_W['Target'].unique()

array([' <=50K', ' >50K', ' <=50K.', ' >50K.'], dtype=object)

In [86]:
dfP_W['Target'] = dfP_W['Target'].str.replace(' ', '', regex=True)
dfP_W['Target'] = dfP_W['Target'].str.replace('.', '', regex=True)
dfP_W['Target'].unique()

array(['<=50K', '>50K'], dtype=object)

### 3.-  Para comparar a personas con salarios mayores a 50k voy a filtrar el dataset y a agruparlo por sexo, raza y educación

In [94]:
dfMayor50k = dfP_W[dfP_W['Target'] == '>50K']
#Agrego una columna contador para poder contar los únicos
dfMayor50k['Counter'] = 1
#Cuento todos los que cobran más de 50k
dfMayor50k['Counter'].sum()


C:\Users\agust\AppData\Local\Temp/ipykernel_23908/450298758.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfMayor50k['Counter'] = 1


11687

In [104]:
# cuento por sexo
dfCountBySex = dfMayor50k['Counter'].groupby([dfP_W['Sex']]).sum()
dfCountBySex


Sex
 Female    1769
 Male      9917
Name: Counter, dtype: int64

In [105]:
# cuento por Raza
dfCountByRace = dfMayor50k['Counter'].groupby([dfP_W['Race']]).sum()
dfCountByRace

Race
 Amer-Indian-Eskimo       55
 Asian-Pac-Islander      409
 Black                   566
 Other                    50
 White                 10606
Name: Counter, dtype: int64

In [106]:
# cuento por Educación
dfCountByEducation = dfMayor50k['Counter'].groupby([dfP_W['Education']]).sum()
dfCountByEducation

Education
 10th              87
 11th              92
 12th              48
 1st-4th            8
 5th-6th           27
 7th-8th           62
 9th               41
 Assoc-acdm       413
 Assoc-voc        522
 Bachelors       3312
 Doctorate        431
 HS-grad         2503
 Masters         1459
 Preschool          1
 Prof-school      617
 Some-college    2063
Name: Counter, dtype: int64